# BTVN 4

In [92]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

## Bài 1

### Câu a

In [93]:
qc = QuantumCircuit(3)

# 1. Chuẩn bị |111>
qc.x([0, 1, 2])

# 2. Mạch phản xạ (CCZ)
qc.ccz(0, 1, 2)

# 3. Check
print("State (a):", Statevector(qc))

State (a): Statevector([ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
             -1.+0.j],
            dims=(2, 2, 2))


### Câu b

In [94]:
qc = QuantumCircuit(3)

# 1. Chuẩn bị |000>

# 2. Mạch phản xạ: X -> CCZ -> X
qc.x([0, 1, 2])
qc.ccz(0, 1, 2)
qc.x([0, 1, 2])

# 3. Check
print("State (b):", Statevector(qc))

State (b): Statevector([-1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
              0.+0.j],
            dims=(2, 2, 2))


### Câu c

In [95]:
qc = QuantumCircuit(3)

# 1. Chuẩn bị |+++>
qc.h([0, 1, 2])

# 2. Mạch phản xạ: H -> X -> CCZ -> X -> H
qc.h([0, 1, 2])
qc.x([0, 1, 2])
qc.ccz(0, 1, 2)
qc.x([0, 1, 2])
qc.h([0, 1, 2])

# 3. Check
print("State (c):", Statevector(qc))

State (c): Statevector([-0.35355339+0.j, -0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j, -0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j, -0.35355339+0.j],
            dims=(2, 2, 2))


### Câu d

In [96]:
qc = QuantumCircuit(3)

# 1. Chuẩn bị |---> (X rồi H)
qc.x([0, 1, 2])
qc.h([0, 1, 2])

# 2. Mạch phản xạ: H -> CCZ -> H
qc.h([0, 1, 2])
qc.ccz(0, 1, 2)
qc.h([0, 1, 2])

# 3. Check
print("State (d):", Statevector(qc))

State (d): Statevector([-0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
             -0.35355339+0.j,  0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j,  0.35355339+0.j],
            dims=(2, 2, 2))


## Câu 3

In [97]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np

def build_r0_circuit():
    # 3 qubit dữ liệu + 1 qubit ancilla (index 3)
    qc = QuantumCircuit(4)
    
    # 1. Chuẩn bị Ancilla về trạng thái |->
    qc.x(3)
    qc.h(3)
    
    # 2. Cài đặt mạch Logic (De Morgan OR)
    # Bước A: Đảo bit (X) để phát hiện trạng thái 000
    qc.x([0, 1, 2])
    
    # Bước B: Multi-controlled X (Kickback phase)
    # Kích hoạt khi q0=q1=q2=1 (tức là input gốc là 000)
    qc.mcx([0, 1, 2], 3)
    
    # Bước C: Đảo bit lại (Uncompute)
    qc.x([0, 1, 2])
    
    # 3. Trả Ancilla về |0> (Dọn dẹp)
    qc.h(3)
    qc.x(3)
    
    return qc

# --- KIỂM TRA KẾT QUẢ ---

# Lấy toán tử của mạch
circuit = build_r0_circuit()
state = Statevector(circuit)

print("Kiểm tra pha của các trạng thái:")

# 1. Kiểm tra trạng thái |000> (Index 0)
amp_000 = state.data[0] 
print(f"Input |000>: {amp_000.real:.2f}")

# 2. Kiểm tra trạng thái |001> (Index 1)
qc_test = QuantumCircuit(4)
qc_test.x(0) # Tạo input 001
qc_test.append(circuit, [0,1,2,3])
amp_001 = Statevector(qc_test).data[1] # Lấy biên độ tại index 1
print(f"Input |001>: {amp_001.real:.2f}")

Kiểm tra pha của các trạng thái:
Input |000>: -1.00
Input |001>: 1.00


## Câu 4

In [98]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

def grover():
    n = 4
    qc = QuantumCircuit(n)

    # 1. Khởi tạo |s>
    qc.h(range(n))
    
    # Số lần lặp tối ưu cho N=16 là 3
    for _ in range(3):
        # --- ORACLE: Mark |0000> ---
        # (X -> MCZ -> X)
        qc.x(range(n))
        qc.mcp(np.pi, list(range(n-1)), n-1)
        qc.x(range(n))
        
        # --- DIFFUSER: Reflect |s> ---
        # (H -> X -> MCZ -> X -> H)
        qc.h(range(n))
        qc.x(range(n))
        qc.mcp(np.pi, list(range(n-1)), n-1)
        qc.x(range(n))
        qc.h(range(n))

    return qc

# --- CHẠY VÀ KIỂM TRA ---

# 1. Tạo mạch
qc = grover()

# 2. Tính toán Statevector
final_state = Statevector(qc)

# 3. Lấy mẫu kết quả
counts = final_state.sample_counts(shots=1000)

# 4. In kết quả
print("Kết quả")
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

for state, count in sorted_counts:
    prob = (count / 1000) * 100
    print(f"Trạng thái |{state}>: {count} lần ({prob:.1f}%)")

# Lấy biên độ chính xác của |0000>
amp_0000 = final_state.data[0]
prob_theoretical = np.abs(amp_0000)**2 * 100
print("-" * 30)
print(f"Xác suất lý thuyết của |0000>: {prob_theoretical:.2f}%")

Kết quả
Trạng thái |0000>: 967 lần (96.7%)
Trạng thái |0101>: 5 lần (0.5%)
Trạng thái |0111>: 5 lần (0.5%)
Trạng thái |1101>: 4 lần (0.4%)
Trạng thái |1000>: 3 lần (0.3%)
Trạng thái |0001>: 2 lần (0.2%)
Trạng thái |0010>: 2 lần (0.2%)
Trạng thái |0100>: 2 lần (0.2%)
Trạng thái |1001>: 2 lần (0.2%)
Trạng thái |1011>: 2 lần (0.2%)
Trạng thái |1100>: 2 lần (0.2%)
Trạng thái |0110>: 1 lần (0.1%)
Trạng thái |1010>: 1 lần (0.1%)
Trạng thái |1110>: 1 lần (0.1%)
Trạng thái |1111>: 1 lần (0.1%)
------------------------------
Xác suất lý thuyết của |0000>: 96.13%


## Câu 5

In [101]:
from qiskit.circuit.library import QFT

def qft_3_qubit():    
    qc = QuantumCircuit(3)
    
    # --- Thiết kế thủ công (Manual Design) ---
    # 1. H trên q2 (tương ứng j1 trong bài làm)
    qc.h(2)
    # Controlled-Phase từ q1 lên q2 (góc pi/2)
    qc.cp(np.pi/2, 1, 2)
    # Controlled-Phase từ q0 lên q2 (góc pi/4)
    qc.cp(np.pi/4, 0, 2)
    
    # 2. H trên q1 (tương ứng j2)
    qc.h(1)
    # Controlled-Phase từ q0 lên q1 (góc pi/2)
    qc.cp(np.pi/2, 0, 1)
    
    # 3. H trên q0 (tương ứng j3)
    qc.h(0)
    
    # 4. SWAP đầu cuối
    qc.swap(0, 2)
    
    return qc

# --- KIỂM TRA ---
# 1. Tạo mạch thủ công
manual_qc = qft_3_qubit()

# 2. Tạo mạch chuẩn thư viện để so sánh
library_qc = QFT(num_qubits=3, do_swaps=True).decompose()

# 3. Tính toán statevector (Ma trận toán tử)
# Ta kiểm tra xem 2 mạch có tạo ra cùng một toán tử Unitary không
sv_manual = Statevector(manual_qc)
sv_lib = Statevector(library_qc) # Input mặc định là |000>

# Tính độ tương đồng (Fidelity)
fidelity = sv_manual.inner(sv_lib)
print(f"Độ trùng khớp (Fidelity): {abs(fidelity):.5f}")

# In thử kết quả với input |000>
print("Statevector output:")
print(np.round(sv_manual.data, 3))

Độ trùng khớp (Fidelity): 1.00000
Statevector output:
[0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j
 0.354+0.j]


/tmp/ipykernel_487187/2374599221.py:32: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  library_qc = QFT(num_qubits=3, do_swaps=True).decompose()
